In [1]:
import toyplot
import numpy as np
from collections import defaultdict

In [2]:
def build_kmers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

In [3]:
#lexicographic order
def sort_kmers(kmers):
  #performing bubble sort 
    n = len(kmers) 
    
    # Traverse through all array elements 
    for i in range(n): 

        # Last i elements are already in place 
        for j in range(0, n-i-1): 

            # traverse the array from 0 to n-i-1 
            # Swap if the element found is greater 
            # than the next element 
            if kmers[j] > kmers[j+1] : 
                kmers[j], kmers[j+1] = kmers[j+1], kmers[j] 
    return kmers

In [4]:
#breaking kmers into left and right mers
#creating graph nodes and edges
def de_bruijn_ize(a) :
    r=[]
    l=[]
    edges = []
    nodes = set()
    AdjList = defaultdict(list)
    
    #breaking kmers into left & right mers
    for i in range(len(a)):
        n1=len(a[i])
        r.append(a[i][1 :n1])
        l.append(a[i][0 :n1-1])
        
    #adding edges and nodes  
    for j in range(len(a)):
        edges.append((l[j],r[j]))
        nodes.add(l[j])
        nodes.add(r[j])
    
    for kmer in range(len(a)):
        head = l[kmer]
        tail = r[kmer]
        AdjList[head].append(tail)
        
    return r, l, nodes, edges, AdjList

In [5]:
#Function to create adjacency matrix and finding starting , ending nodes 
def Adj_Mat(a, l, r):
    #a is nodes
    # converting set(nodes) to list
    a = list(a)
    Adj = np.zeros((len(a),len(a)))
    for i in range(len(a)):
        for j in range(len(a)):
            #Mapping done based on directed graphs
            if(a[i] != a[j] and l[j] == r[i]):
                Adj[i][j] = Adj[i][j]+1
    
    s=''
    #starting node
    starting='' 
    e=''
    #ending node
    ending=''
    outdegree = Adj.sum(axis = 1)
    indegree = Adj.sum(axis = 0)
    count1 = 0
    count2 = 0
    for i in range(len(outdegree)):
        #Obtaining the ending point as it's outdegree is minimum, i.e, 0(ideal)
        if(outdegree[i] == min(outdegree) and count1 < 1):
            e = e + str(a[i])
            count1 = count1 + 1
        #Obtaining the starting point as it's indegree is minimum, i.e, 0(ideal) 
        if(indegree[i] == min(indegree) and count2 < 1):
            s = s + str(a[i])
            count2 = count2 + 1
    
    for z in range(len(s)):
        starting = str(s[0:z])
    for y in range(len(e)):
        ending = str(e[1:y+1])
    return starting, ending, Adj

In [6]:
def plot_debruijn_graph(edges, v, width=1100, height=700):
    
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">", 
        vsize=v,
        vstyle={"stroke": "black", "stroke-width": 2, "fill": "none"},
        vlstyle={"font-size": "12px"},
        estyle={"stroke": "black", "stroke-width": 3},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.StraightEdges()))
    return graph

In [7]:
def Debrujin_graph(seqence, kmer_len):
    k=(build_kmers(seqence, kmer_len))
    print("Kmers of the given string are")
    print(k, '\n')
    lexio=sort_kmers(k)
    print("Sorted Kmers of the given string TTGAGTGCCAGCG ")
    print(lexio, '\n')
    r, l, nodes, edges, A = de_bruijn_ize(lexio)
    print("Nodes of the graph ")
    print(nodes, '\n')
    print("Nodes which form the edges of the graph ")
    print(edges, '\n')
    print("Adjacency list of the graph ")
    print(A, '\n')
    start, ending, adj=Adj_Mat(lexio, l, r)
    print("starting node:")
    print(start, '\n')
    print("ending node:")
    print(ending, '\n')
    print("Debrujin Graph:")
    plot_debruijn_graph(edges, v=float(kmer_len*7))
    

In [8]:
Debrujin_graph("TAATGCCATGGGATGTT",3)

Kmers of the given string are
['TAA', 'AAT', 'ATG', 'TGC', 'GCC', 'CCA', 'CAT', 'ATG', 'TGG', 'GGG', 'GGA', 'GAT', 'ATG', 'TGT', 'GTT'] 

Sorted Kmers of the given string TTGAGTGCCAGCG 
['AAT', 'ATG', 'ATG', 'ATG', 'CAT', 'CCA', 'GAT', 'GCC', 'GGA', 'GGG', 'GTT', 'TAA', 'TGC', 'TGG', 'TGT'] 

Nodes of the graph 
{'TA', 'TG', 'CA', 'GC', 'CC', 'TT', 'GT', 'AT', 'GG', 'GA', 'AA'} 

Nodes which form the edges of the graph 
[('AA', 'AT'), ('AT', 'TG'), ('AT', 'TG'), ('AT', 'TG'), ('CA', 'AT'), ('CC', 'CA'), ('GA', 'AT'), ('GC', 'CC'), ('GG', 'GA'), ('GG', 'GG'), ('GT', 'TT'), ('TA', 'AA'), ('TG', 'GC'), ('TG', 'GG'), ('TG', 'GT')] 

Adjacency list of the graph 
defaultdict(<class 'list'>, {'AA': ['AT'], 'AT': ['TG', 'TG', 'TG'], 'CA': ['AT'], 'CC': ['CA'], 'GA': ['AT'], 'GC': ['CC'], 'GG': ['GA', 'GG'], 'GT': ['TT'], 'TA': ['AA'], 'TG': ['GC', 'GG', 'GT']}) 

starting node:
TA 

ending node:
TT 

Debrujin Graph:


<svg class="toyplot-canvas-Canvas" height="700.0px" id="ta14fed6217794f7a98c65eebe6398168" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1100.0 700.0" width="1100.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> AA AT CA CC GA GC GG GT TA TG TT

In [9]:
Debrujin_graph("TAATGCCATGGGATGTT",4)

Kmers of the given string are
['TAAT', 'AATG', 'ATGC', 'TGCC', 'GCCA', 'CCAT', 'CATG', 'ATGG', 'TGGG', 'GGGA', 'GGAT', 'GATG', 'ATGT', 'TGTT'] 

Sorted Kmers of the given string TTGAGTGCCAGCG 
['AATG', 'ATGC', 'ATGG', 'ATGT', 'CATG', 'CCAT', 'GATG', 'GCCA', 'GGAT', 'GGGA', 'TAAT', 'TGCC', 'TGGG', 'TGTT'] 

Nodes of the graph 
{'CAT', 'CCA', 'GGG', 'GTT', 'TGT', 'TGG', 'AAT', 'GCC', 'GAT', 'TGC', 'TAA', 'ATG', 'GGA'} 

Nodes which form the edges of the graph 
[('AAT', 'ATG'), ('ATG', 'TGC'), ('ATG', 'TGG'), ('ATG', 'TGT'), ('CAT', 'ATG'), ('CCA', 'CAT'), ('GAT', 'ATG'), ('GCC', 'CCA'), ('GGA', 'GAT'), ('GGG', 'GGA'), ('TAA', 'AAT'), ('TGC', 'GCC'), ('TGG', 'GGG'), ('TGT', 'GTT')] 

Adjacency list of the graph 
defaultdict(<class 'list'>, {'AAT': ['ATG'], 'ATG': ['TGC', 'TGG', 'TGT'], 'CAT': ['ATG'], 'CCA': ['CAT'], 'GAT': ['ATG'], 'GCC': ['CCA'], 'GGA': ['GAT'], 'GGG': ['GGA'], 'TAA': ['AAT'], 'TGC': ['GCC'], 'TGG': ['GGG'], 'TGT': ['GTT']}) 

starting node:
TAA 

ending node:
GTT 

Deb

<svg class="toyplot-canvas-Canvas" height="700.0px" id="t4865c8b5ea344425ae375944937e3534" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1100.0 700.0" width="1100.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> AAT ATG CAT CCA GAT GCC GGA GGG GTT TAA TGC TGG TGT

In [10]:
Debrujin_graph("GCGGTAATGCAGTTGAC",5)

Kmers of the given string are
['GCGGT', 'CGGTA', 'GGTAA', 'GTAAT', 'TAATG', 'AATGC', 'ATGCA', 'TGCAG', 'GCAGT', 'CAGTT', 'AGTTG', 'GTTGA', 'TTGAC'] 

Sorted Kmers of the given string TTGAGTGCCAGCG 
['AATGC', 'AGTTG', 'ATGCA', 'CAGTT', 'CGGTA', 'GCAGT', 'GCGGT', 'GGTAA', 'GTAAT', 'GTTGA', 'TAATG', 'TGCAG', 'TTGAC'] 

Nodes of the graph 
{'CGGT', 'TGAC', 'AGTT', 'TGCA', 'GTAA', 'GGTA', 'GCAG', 'GTTG', 'CAGT', 'ATGC', 'AATG', 'GCGG', 'TAAT', 'TTGA'} 

Nodes which form the edges of the graph 
[('AATG', 'ATGC'), ('AGTT', 'GTTG'), ('ATGC', 'TGCA'), ('CAGT', 'AGTT'), ('CGGT', 'GGTA'), ('GCAG', 'CAGT'), ('GCGG', 'CGGT'), ('GGTA', 'GTAA'), ('GTAA', 'TAAT'), ('GTTG', 'TTGA'), ('TAAT', 'AATG'), ('TGCA', 'GCAG'), ('TTGA', 'TGAC')] 

Adjacency list of the graph 
defaultdict(<class 'list'>, {'AATG': ['ATGC'], 'AGTT': ['GTTG'], 'ATGC': ['TGCA'], 'CAGT': ['AGTT'], 'CGGT': ['GGTA'], 'GCAG': ['CAGT'], 'GCGG': ['CGGT'], 'GGTA': ['GTAA'], 'GTAA': ['TAAT'], 'GTTG': ['TTGA'], 'TAAT': ['AATG'], 'TGCA': ['GCA

<svg class="toyplot-canvas-Canvas" height="700.0px" id="t432b73653e3f48ac85d836bbba5a65f0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1100.0 700.0" width="1100.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> AATG AGTT ATGC CAGT CGGT GCAG GCGG GGTA GTAA GTTG TAAT TGAC TGCA TTGA